In [ ]:
from openai import OpenAI
client = OpenAI()

# Agente de contrução do relatório:

### Funções Auxiliares para o prompt:

In [8]:
# Gerando a data de hoje em São Paulo, Brasil
from datetime import datetime
from datetime import timedelta

data_de_hoje = datetime.now().strftime("%d/%m/%Y")

# Gerando a data de ontem em São Paulo, Brasil
ontem = datetime.now() - timedelta(days=1)
data_de_ontem = ontem.strftime("%d/%m/%Y")

print(f"Data de hoje: {data_de_hoje}")
print(f"Data de ontem: {data_de_ontem}")

Data de hoje: 21/05/2025
Data de ontem: 20/05/2025


In [ ]:
import pandas as pd
import requests
import datetime
import numpy as np
import yfinance as yf
import json


# Função para coletar indicadores diários do Bitcoin usando a API do Yahoo Finance
def get_bitcoin_data_yf():
    """
    Coleta indicadores referentes ao Bitcoin usando a API pública do Yahoo Finance.

    Retorno:
    - Dicionário:
        'descrição': 'description'
        'preço_anterior': 'previousClose'
        'preço_atual': 'regularmarketPrice'
        'variação_24h': 'regularmarketChange'
        'variação_percentual_24h': 'regularmarketChangePercent'
        'máximo_24h': 'dayHigh'
        'mínimo_24h': 'dayLow'
        'volume_24h': 'regularmarketVolume'
        'volume_10_dias': 'averageDailyVolume10Day'
        'volume_3_meses': 'averageDailyVolume3Month'
        'capitalização_mercado': 'marketCap'
        'fornecimento_circulante': 'circulatingSupply'
        'MSA_50': 'fiftyDayAverage'
        'MSA_50_variacao': 'fiftyDayAverageChange'
        'MSA_50_variacao_percentual': 'fiftyDayAverageChangePercent'
        'MSA_200': 'twoHundredDayAverage'
        'MSA_200_variacao': 'twoHundredDayAverageChange'
        'MSA_200_variacao_percentual': 'twoHundredDayAverageChangePercent'
        'faixa_52_semanas_low': 'fiftyTwoWeekLow'
        'faixa_52_semanas_high': 'fiftyTwoWeekHigh'
        'faixa_52_semanas_low_variacao': 'fiftyTwoWeekLowChange'
        'faixa_52_semanas_low_variacao_percentual': 'fiftyTwoWeekLowChangePercent'
        'faixa_52_semanas_high_variacao': 'fiftyTwoWeekHighChange'
        'faixa_52_semanas_high_variacao_percentual': 'fiftyTwoWeekHighChangePercent'
        'faixa_52_semanas_variacao_percentual': 'fiftyTwoWeekChangePercent'    
    """
    # Definir símbolo e intervalo da API Yahoo Finance
    symbol = "BTC-USD"  # Bitcoin vs USD
    # Coletar dados usando yfinance
    raw_data = yf.Ticker(symbol)
    raw_data_dict = raw_data.get_info()
    # Extraindo os dados relevantes do dicionário raw_data_dict
    bitcoin_data = {
        "descrição": raw_data_dict.get("description", 0),
        "preço_anterior": raw_data_dict.get("previousClose", 0),
        "preço_atual": raw_data_dict.get("regularMarketPrice", 0),
        "variação_24h": raw_data_dict.get("regularMarketChange", 0),
        "variação_percentual_24h": raw_data_dict.get("regularMarketChangePercent", 0),
        "máximo_24h": raw_data_dict.get("dayHigh", 0),
        "mínimo_24h": raw_data_dict.get("dayLow", 0),
        "volume_24h": raw_data_dict.get("volume24Hr", 0),
        "volume_10_dias": raw_data_dict.get("averageDailyVolume10Day", 0),
        "volume_3_meses": raw_data_dict.get("averageDailyVolume3Month", 0),
        "capitalização_mercado": raw_data_dict.get("marketCap", 0),
        "fornecimento_circulante": raw_data_dict.get("circulatingSupply", 0),
        "MSA_50": raw_data_dict.get("fiftyDayAverage", 0),
        "MSA_50_variacao": raw_data_dict.get("fiftyDayAverageChange", 0),
        "MSA_50_variacao_percentual": raw_data_dict.get("fiftyDayAverageChangePercent", 0),
        "MSA_200": raw_data_dict.get("twoHundredDayAverage", 0),
        "MSA_200_variacao": raw_data_dict.get("twoHundredDayAverageChange", 0),
        "MSA_200_variacao_percentual": raw_data_dict.get("twoHundredDayAverageChangePercent", 0),
        "faixa_52_semanas_low": raw_data_dict.get("fiftyTwoWeekLow", 0),
        "faixa_52_semanas_high": raw_data_dict.get("fiftyTwoWeekHigh", 0),
        "faixa_52_semanas_low_variacao": raw_data_dict.get("fiftyTwoWeekLowChange", 0),
        "faixa_52_semanas_low_variacao_percentual": raw_data_dict.get("fiftyTwoWeekLowChangePercent", 0),
        "faixa_52_semanas_high_variacao": raw_data_dict.get("fiftyTwoWeekHighChange", 0),
        "faixa_52_semanas_high_variacao_percentual": raw_data_dict.get("fiftyTwoWeekHighChangePercent", 0),
        "faixa_52_semanas_variacao_percentual": raw_data_dict.get("fiftyTwoWeekChangePercent", 0),
    }
    # Retornar os dados coletados
    dados_indicadores_str = json.dumps(bitcoin_data, indent=2, ensure_ascii=False)
    return dados_indicadores_str

# Função para coletar dados históricos do Bitcoin usando a API do Yahoo Finance
def get_historical_data_yf():
    """
    Coleta dados históricos do Bitcoin usando a API pública do Yahoo Finance.

    Retorno:
    - DataFrame do pandas contendo os dados históricos do Bitcoin.
    """
    # Definir símbolo e intervalo da API Yahoo Finance
    symbol = "BTC-USD"  # Bitcoin vs USD
    # Coletar dados históricos usando yfinance
    raw_data = yf.Ticker(symbol)
    df_dados = raw_data.history(period="1y", interval="1d")
    #Deixando apenas as colunas "Date" e "Close"
    df_dados = df_dados.reset_index()
    df_dados = df_dados[['Date', 'Close']]
    #Alterando a coluna "Date" para o formato DD/MM/AAAA
    df_dados['Date'] = df_dados['Date'].dt.strftime('%d/%m/%Y')
    hist_list_of_dicts = df_dados.to_dict(orient='records')
    dados_historicos_str = json.dumps(hist_list_of_dicts, indent=2)
    # Retornar os dados coletados
    return dados_historicos_str


In [ ]:
dados_historicos = get_historical_data_yf()
print(dados_historicos)  # Isso exibirá a lista de dicionários com OHLC dos últimos 90 dias.
print("\n\n")

# dados_indicadores = get_bitcoin_data_yf()
# print(dados_indicadores)  # Isso exibirá a lista de dicionários com OHLC dos últimos 90 dias.
# print("\n\n")

# dados_indicadores_plus = get_indicators()
# print(dados_indicadores_plus)  # Isso exibirá a lista de dicionários com OHLC dos últimos 90 dias. 

In [ ]:
PROMPT_05_04_2025 = """
Você é um analista técnico e estratégico profissional especializado em criptomoedas, com ênfase em Bitcoin. Você tem um amplo conhecimento de mercado financeiro em análises técnicas, fundamentalistas e de sentimento de mercado. Você utiliza seu conhecimento para gerar previsões e considerações estratégicas para investidores, baseado nos dados de mercado.

# TAREFA
- Sua tarefa é elaborar um relatório técnico e estratégico do Bitcoin (BTC) que apresenta indicadores, interpretações e considerações estratégicas, baseado em dados reais e atualizados rereferentes ao dia de hoje, {data_de_hoje}, em São Paulo, Brasil. 
- O relatório deve ser gerado em formato JSON, seguindo um esquema estruturado fornecido (BTCReport). O esquema define a estrutura e os campos obrigatórios do relatório. Você deve preencher todos os campos obrigatórios do esquema com as informações coletadas e analisadas.
- Abaixo estão as instruções detalhadas que você deve seguir para gerar o relatório:
    
# INSTRUÇÕES DETALHADAS PARA GERAR O OBJETO JSON (`BTCReport`)
## 1. LEITURA E COMPREENSÃO
- Revise CUIDADOSAMENTE a estrutura do relatório baseado no esquema fornecido (`BTCReport`), para você entender e listar quais os dados e as informações que serão necessarias para preencher cada item do objeto JSON final.
- Os detalhes, descrições e informações do conteúdo de cada seção do relatório estão descritos em na seção CONTEUDO DO RELATÓRIO JSON (BTCReport schema). Você deve ler cuidadosamente cada item e entender o que é necessário para preencher cada seção do relatório. 
- Para a construção do relatório você deve seguir as intruções detalhadas abaixo:

## 2. COLETA DE DADOS
- Após ler e entender o conteúdo necessário para cada item do relatório e os campos do esquema fornecido, colete **todos** os dados que você julgar necessários para preencher as seções descritas.
- Os dados históricos de preço e os principais indicadores técnicos do Bitcoin são fornecidos na seção DADOS FORNECIDOS. 
- Os indicadores que não são diretamente fornecidos nas seções 2 e 3, devem ser obrigatoriamente calculados por você utilizando os dados fornecidos e seu conhecimento técnico de mercado financeiro.
- Caso necessario, utilize a ferramenta de busca "web_search_tool" para buscar dados adicionais necessários para completar as seções do relatório.
- Faça uma busca na internet para coletar diversas notícias relevantes e recentes (últimas 24-48h) sobre o Bitcoin utilizando a ferramenta de busca "web_search_tool".
- Só avance para a análise após coletar todos os dados externos necessários

## 3. ANÁLISES APROFUNDADAS
- Após revisar a estrutura do relatório e coletar **todos** os dados necessários, você deve inciar suas análises.
- Verfique se os dados coletados estão atualizados e são consistentes entre si. Use a data de hoje, {data_de_hoje}, como referência.
- Faça uma análise cuidadosa sobre os dados coletados para extrair insights significativos e relevantes para preencher os campos do schema JSON.
- Para preencher o conteudo dos campos do relatório JSON (descritos em <CONTEUDO DO RELATÓRIO JSON (BTCReport schema)>) que apresentam "interpretações" e "considerações estratégicas", você deve obrigatóriamente considerar os seguintes aspectos antes de gerar sua resposta:
    
    - **Análise técnica**: observe o comportamento histórico de preços do Bitcoin para detectar padrões, focando no comportamento passado para prever tendências futuras. Utiliza indicadores e ferramentas para tentar entender a psicologia por trás dos movimentos de preços.
    
    - **Análise fundamentalista**: examine fatores econômicos, financeiros e outros dados qualitativos e quantitativos para avaliar o valor intrínseco do Bitcoin. Considera eventos macroeconômicos, notícias relevantes e mudanças no ambiente regulatório.
    
    - **Análise de sentimento**: avalie o sentimento geral do mercado de Bitcoin, utilizando dados de redes sociais, notícias relevantes e outros indicadores para entender a psicologia coletiva dos investidores. Isso pode incluir o uso de índices de Fear & Greed, entre outros.
- Utilize Python para realizar cálculos necessários.
- Faça uma validação cruzada com informações externas confiáveis, para verificar a consistência dos seus resultados. Você pode utilizar a ferramenta de busca "web_search_tool" para buscar realizar essas validações.
- Após realizar suas análises e validações, você deve preencher os campos do objeto JSON com as informações obtidas.

## 4. CONTEUDO DO RELATÓRIO JSON (BTCReport schema)
1. `resumo`: 
    - Utilize **exclusivamente** os indicadores fornecidos na seção <DADOS_FORNECIDOS> como referencia para preencher a seção 1 do relatório.
    - Não arredonde os valores fornceidos, mantenha os valores exatos.
    - Utilize o formato de moeda brasileira (R$) para os valores, com separador de milhares e duas casas decimais (float). 

2. `análise_tecnica`:
    - Calcule (usando os dados históricos fornecidos em <DADOS_FORNECIDOS>) e apresente os valores da SMA 50 e SMA 200. Forneça uma interpretação sobre os valores obtidos e o que eles representam (tendencia de alta ou baixa).
    - Descreva o cruzamento de médias móveis, analizando o histórico das SMAs para identificar e descrever o último cruzamento relevante (Dourado/Morte) ou a ausência dele.
    - Apresente o valor do RSI(14) e sua interpretação (sobrecompra/venda/neutro) baseado nas suas análises.
    - Calcule as Bandas de Bollinger baseado nos dados coletados e descreva a posição atual do preço e a situação das bandas (estreitas/largas). Indique o que isso sugere sobre a volatilidade e a pressão compradora/vendedora.
    - As interpretações das análises técnicas devem ter pelo menos 1 linha de texto cada.

3. `suporte_resistência`:
    - Liste os **três** níveis de suporte e **três** de resistência identificados, com seus valores e uma breve interpretação técnica para cada.
    - Considere os seguintes aspectos na sua análise: Pontos de preço onde a cotação encontrou mínimas (suporte) ou máximas (resistência), identificação de linhas de tendência, análise de séries históricas de preços, uso de indicadores técnicos, consideração do volume de negociação, tempo e amplitude da congestão, etc.
    - As interpretações técnicas de cada nível deve ter pelo menos 2 linhas de texto cada.

4. `notícias`:
    - Apresente obrigatóriamente **duas** notícias relevantes encontradas relacionadas ao Bitcoin, incluindo um resumo e o link. As notícias devem ser **OBRIGATÓRIAMENTE** do período de {data_de_ontem} até hoje {data_de_hoje}.

5. `estrategias`:
    - Formule considerações estratégicas para compradores e vendedores baseando-se nos indicadores, nas aspectos das suas analises técnica, fundamentalista e de sentimento de mercado para fornecer uma resposta mais precisa, objetiva e fundamentada.
    - `compradores`: Gere uma string com sugestões objetivas e condicionais para potenciais pontos de entrada ou zonas de interesse, mencionando alvos baseados em resistências. 
    - `vendedores`: Gere uma string com sugestões objetivas e condicionais sobre áreas potenciais de realização de lucro (próximo a resistências) e níveis para considerar stop-loss (abaixo de suportes).
    - As sugestões devem ter pelo menos 3 linhas de texto cada.

6. `conclusão`:
    - Gere um resumo final da tendência de curto prazo, principais riscos e pontos de atenção identificados na análise geral (técnica, fundamentalista, sentimento).
    - A conclusão deve ter pelo menos 3 linhas de texto.
    
# DADOS FORNECIDOS
Estes são os dados base que você deve utilizar:

<Dados históricos do preço do Bitcoin>
{dados_historicos_binance}
</Dados históricos do preço do Bitcoin>

<Indicadores técnicos do Bitcoin>
{dados_indicadores}
</Indicadores técnicos do Bitcoin>

<Notícias relevantes>
{noticias}
</Notícias relevantes>

# VALIDAÇÃO
- OUTPUT EXCLUSIVAMENTE JSON: Garanta que a saída final seja *apenas* o objeto JSON válido.
- ADESÃO ESTRITA AO SCHEMA: O JSON gerado deve corresponder *exatamente* à estrutura `BTCReport`. Todos os campos definidos no schema são obrigatórios.
- Confirme que todos os dados do relatório (indicadores, interpretações e considerações estratégicas) foram preenchidos corretamente.
- Verifique se as respostas fornecidas estão de acordo com a estrutura e formatação fornecida.
- Mantenha a linguagem clara e profissional, e não faça suposições não fundamentadas.
"""

In [ ]:
prompt_formatado = PROMPT_05_04_2025.format(
    data_de_hoje=data_de_hoje,
    data_de_ontem=data_de_ontem,
    dados_historicos_binance=dados_historicos_binance,
    dados_indicadores=dados_indicadores
)

In [ ]:
print(prompt_formatado)

<FORMATO DO RELATÓRIO>
- O relatório deve ser escrito em português. 
- O relatório deve ser gerado em formato Markdown.
- O relatório deve ser dividido em seções, conforme descrito na seção "Conteúdo de cada seção do relatório".
- Apresente os dados com blocos bem separados e emojis temáticos no início de cada seção (ex: 📊, 🔍, 📈, 📰, 💡, ✅, ⚠️).
- Os valores devem estar sempre em dólares (USD) com formatação correta (ex: $80.000,00 USD).
- Adicione sempre ao final do relatório uma seção de aviso com a seguinte mensagem: "⚠️ Aviso: Ao realizar operações de compra ou venda, é importante estar ciente de que o preço do Bitcoin pode sofrer variações devido a movimentos de mercado e notícias que possam impactar seu valor"
- Ao gerar o Markdown final, no corpo do relatório, não exiba marcações de citação, notas de rodapé nem URLs, com exceção dos hyperlinks das duas notícias listadas na seção correspondente.
- O relatório deve seguir EXATAMENTE o formato indicado.
</FORMATO DO RELATÓRIO>


<CONTEÚDO DE CADA SEÇÃO DO RELATÓRIO>
1. Resumo de mercado, com os seguintes dados:
    - **Preço atual do Bitcoin em dólares (USD)**
    - **Variação percentual nas últimas 24 horas**
    - **Valor máximo nas últimas 24 horas**
    - **Valor mínimo nas últimas 24 horas**
    - **Volume de negociação nas últimas 24 horas**
    - **Capitalização de mercado total do Bitcoin**
    - **Fornecimento total de Bitcoin em circulação**
    - **Fornecimento máximo de Bitcoin**

2. Análise técnica do mercado de Bitcoin, com os seguintes indicadores:
    - **Média Móvel Simples (SMA) de 50 dias**: valor atual e interpretação (tendência de alta ou baixa).
    - **Média Móvel Simples (SMA) de 200 dias**: valor atual e interpretação (tendência de alta ou baixa).
    - **Cruzamento de Médias Móveis**: se a SMA de 50 dias cruzou acima ou abaixo da SMA de 200 dias e o que isso indica (sinal de compra ou venda).   
    - **Índice de Força Relativa (RSI)**: valor atual e interpretação (neutro, sobrecompra ou sobrevenda).
    - **Bandas de Bollinger**: posição do preço em relação às bandas e o que isso indica (volatilidade, pressão compradora ou vendedora).

3. Níveis de Suporte e Resistência Atualizados:
    - **Suportes**: três níveis de suporte com valores e interpretação.
    - **Resistências**: três níveis de resistência com valores e interpretação.

4. Notícias Relevantes:
    - **Notícia 1**: notícia referente ao Bitcoin, com link para a fonte e resumo do conteúdo.
    - **Notícia 2**: notícia referente ao Bitcoin, com link para a fonte e resumo do conteúdo.

5. Considerações Estratégicas:
    - **Para compradores**: sugestão objetiva para compradores com foco em ponto de entrada e alvo, com base nos dados técnicos e fundamentais.
    - **Para vendedores**: sugestão objetiva para vendedores com foco em resistência, pontos de realização de lucro, riscos e stop loss, com base nos dados técnicos e fundamentais.

6. Conclusão: 
    - Resumo da tendência de curto prazo, principais riscos, pontos de atenção e recomendação geral.

7. Aviso:
    - Aviso padrão: "⚠️ Aviso: Ao realizar operações de compra ou venda, é importante estar ciente de que o preço do Bitcoin pode sofrer variações devido a movimentos de mercado e notícias que possam impactar seu valor."
</CONTEÚDO DE CADA SEÇÃO DO RELATÓRIO>


## 7. ESTRATÉGIA DE RACIOCÍNIO
1. Verifique a estrutura geral e os itens obrigatórios do relatório em "Conteúdo de cada seção do relatório" e faça uma lista de todos os dados que você precisa coletar. Pense cuidadosamente sobre quais desses dados são objetivos (dados diretos) e quais são subjetivos (dados que necessitam de análise).
2. Faça a coleta na internet de todas as informações necessárias para preencher o relatório.
    a. Verifique se os dados coletados estão atualizados e são de fontes confiáveis.
3. Preencha os dados objetivos do relatório com as informações coletadas.
4. Preencha os itens subjetivos do relatório fazendo as análises técnicas necessárias, utilizando os dados objetivos coletados, as notícias relevantes e seu conhecimento técnico sobre o mercado financeiro e criptomoedas.
5. Após pensar cuidadosamente se os dados estão corretos e se as análises estão completas, gere o relatório final seguindo a estrutura e o formato definidos.
6. Revise o relatório final, verificando se está claro, objetivo e se segue todas as instruções dadas.
    a. Caso encontre erros ou inconsistências, volte para o passo (2.) e faça as correções necessárias.


Após concluir a análise técnica por conta própria, você deve realizar uma validação cruzada com informações externas confiáveis, para verificar se suas conclusões estão alinhadas com a visão de outros analistas ou dados públicos recentes. Siga estes passos:
    1. Obtendo Referências Externas: Caso tenha acesso a análises de mercado atuais, notícias relevantes ou relatórios públicos (por exemplo, previsões de analistas reconhecidos, notícias macroeconômicas que afetem o Bitcoin, etc.), reúna essas informações.

    2. Consistência de Resultados: Compare os pontos-chave da sua análise técnica (tendência identificada, níveis críticos de suporte/resistência, sinais de indicadores) com o que dizem as fontes externas.

    3. Fontes Confiáveis: Ao citar ou usar informações externas, prefira fontes respeitáveis e recentes. Exemplos: portais especializados (CoinDesk, Bloomberg, etc.), opiniões de traders renomados, dados on-chain atualizados, ou comunicados econômicos relevantes. Evite rumores ou fontes não verificadas. 
    
    4. Validação Qualitativa: Além de comparar números, verifique se o sentimento de mercado descrito externamente (otimismo, medo, etc.) condiz com o que os dados técnicos estão mostrando. Isso adiciona uma camada estratégica: por exemplo, se os dados técnicos indicam alta mas as notícias são pessimistas, o investidor deve ter cautela, e vice-versa.

<Exemplo de Relatório>
```markdown
📌 **Resumo de Mercado - 17 de abril de 2025**
    • Preço Atual: US$ 84.411,00
    • Variação 24h: +0,62%
    • Máxima 24h: US$ 85.312,00
    • Mínima 24h: US$ 83.536,00
    • Volume 24h: US$ 27,13 bilhões
    • Capitalização de Mercado: US$ 1,68 trilhão
    • Fornecimento Circulante: 19,85 milhões BTC
    • Fornecimento Máximo: 21 milhões BTC

📊 **Análise Técnica**
    • Médias Móveis (MA):
    • MA 50 dias: US$ 85.000 (atuando como resistência dinâmica)
    • MA 200 dias: US$ 80.000 (servindo como suporte de longo prazo)
    • Cruzamento de Médias: Ocorreu recentemente um “cruzamento dourado”, onde a MA de 50 dias cruzou acima da MA de 200 dias, sugerindo uma tendência de alta de longo prazo.
    • Índice de Força Relativa (RSI):
    • Atualmente em 68, indicando que o ativo está se aproximando da zona de sobrecompra.
    • Bandas de Bollinger:
    • As bandas estão se estreitando, sinalizando uma redução na volatilidade e possivelmente antecipando um movimento de preço significativo.

🔍 **Níveis de Suporte e Resistência**
    • Suportes:
    • US$ 83.900: Suporte imediato observado recentemente.
    • US$ 83.061: Próximo nível de suporte técnico.
    • US$ 82.478: Alinhado com a média móvel de 200 períodos, servindo como um suporte chave.
    • Resistências:
    • US$ 84.500 - US$ 84.700: Zona de resistência onde o preço enfrentou rejeições.
    • US$ 86.200: Próximo nível de resistência observado.
    • US$ 88.600: Resistência adicional a ser monitorada.

📰 **Notícias Relevantes**
    • Bitcoin abaixo de US$ 85 mil causa saídas de US$ 171 milhões de ETFs: O Bitcoin tem enfrentado dificuldade significativa em superar a marca de US$ 85 mil nesta semana, e seu preço permanece estagnado abaixo dessa resistência chave. Assim, os entusiastas do BTC estão cada vez mais frustrados. Isso porque a criptomoeda luta para manter o impulso ascendente. Junto com essa estagnação de preço, houve uma queda no interesse aberto e saídas de ETFs, refletindo a crescente incerteza no mercado.

💡 **Considerações Estratégicas**
    • Para Compradores:
    • Considerar entradas próximas ao suporte de US$ 83.000, visando uma possível valorização até a resistência de US$ 85.500.
    • Para Vendedores:
    • Observar a resistência em US$ 85.500 como um possível ponto de realização de lucros, ajustando stops conforme a volatilidade do mercado.

✅ Conclusão
    O Bitcoin apresenta uma leve tendência de alta no curto prazo, com suporte em US$ 83.000 e resistência em US$ 85.500. Fatores externos, como políticas econômicas e eventos globais, continuam influenciando o mercado. Recomenda-se monitorar os níveis de suporte e resistência mencionados e estar atento às notícias que possam impactar o valor do ativo.

⚠️ Aviso: Ao realizar operações de compra ou venda, é importante estar ciente de que o preço do Bitcoin pode sofrer variações devido a movimentos de mercado e notícias que possam impactar seu valor.
```
</Exemplo de Relatório>

In [ ]:
# Criando uma classe com esquema Pydantic com os valores do relatório
from pydantic import BaseModel, Field

class Resumo_de_Mercado(BaseModel):
    preco_atual: float 
    variacao_24h_pct: float
    max_24h: float
    min_24h: float
    volume_24h: float
    market_cap: float
    fornecimento_circulante: float
    fornecimento_max: float

class Analise_Tecnica(BaseModel):
    sma50: float = Field(..., description="valor da SMA 50")
    sma50_interpretacao: str = Field(..., description="interpretação da SMA 50")
    sma200: float = Field(..., description="valor da SMA 200")
    sma200_interpretacao: str = Field(..., description="interpretação da SMA 200")
    cruzamento_medias_interpretacao: str = Field(..., description="interpretação do cruzamento de médias móveis")
    rsi: float = Field(..., description="valor do RSI")
    rsi_interpretacao: str = Field(..., description="interpretação do RSI")
    bollinger_interpretacao: str = Field(..., description="interpretação das Bandas de Bollinger")                          

class Nivel_Suporte(BaseModel):
    nivel_suporte_valor: float = Field(..., description="valor do nível de suporte calculado")
    nivel_suporte_interpretacao: str = Field(..., description="interpretação do nível de suporte calculado")

class Nivel_Resistencia(BaseModel):
    nivel_resistencia_valor: float = Field(..., description="valor do nível de resistencia calculado")
    nivel_resistencia_interpretacao: str = Field(..., description="interpretação do nível de resistencia calculado")

class Suporte_Resistencia(BaseModel):
    suporte1: Nivel_Suporte
    suporte2: Nivel_Suporte
    suporte3: Nivel_Suporte
    resistencia1: Nivel_Resistencia
    resistencia2: Nivel_Resistencia
    resistencia3: Nivel_Resistencia

class Noticia(BaseModel):
    titulo: str = Field(..., description="título da notícia", examples="Preço do Bitcoin atinge novo recorde")
    link: str = Field(..., description="link da notícia", examples="https://www.binance.com/bitcoin-news")
    resumo: str = Field(..., description="resumo da notícia", examples="O preço do Bitcoin atingiu um novo recorde histórico de US$ 100.000, impulsionado por uma onda de compras institucionais.")

class Noticias(BaseModel):
    noticia1: Noticia
    noticia2: Noticia

class Estrategias(BaseModel):
    compradores: str = Field(..., description="considerações estratégicas para compradores", example="Considerar entradas próximas ao suporte de US$ 83.000, visando uma possível valorização até a resistência de US$ 85.500.")
    vendedores: str = Field(..., description="considerações estratégicas para vendedores", example="Observar a resistência em US$ 85.500 como um possível ponto de realização de lucros, ajustando stops conforme a volatilidade do mercado.")

class BTCReport(BaseModel):
    resumo: Resumo_de_Mercado
    analise_tecnica: Analise_Tecnica
    suporte_resistencia: Suporte_Resistencia
    noticias: Noticias
    estrategias: Estrategias
    conclusao: str


In [ ]:
# Criando uma classe com esquema Pydantic com os valores do relatório
from pydantic import BaseModel, Field

class Resumo_de_Mercado(BaseModel):
    preco_atual: float 
    variacao_24h_pct: float
    max_24h: float
    min_24h: float
    volume_24h: float
    market_cap: float
    fornecimento_circulante: float
    fornecimento_max: float

class Analise_Tecnica(BaseModel):
    sma50: float 
    sma50_interpretacao: str 
    sma200: float 
    sma200_interpretacao: str 
    cruzamento_medias_interpretacao: str 
    rsi: float 
    rsi_interpretacao: str 
    bollinger_interpretacao: str                           

class Nivel_Suporte(BaseModel):
    nivel_suporte_valor: float 
    nivel_suporte_interpretacao: str 

class Nivel_Resistencia(BaseModel):
    nivel_resistencia_valor: float 
    nivel_resistencia_interpretacao: str 

class Suporte_Resistencia(BaseModel):
    suporte1: Nivel_Suporte
    suporte2: Nivel_Suporte
    suporte3: Nivel_Suporte
    resistencia1: Nivel_Resistencia
    resistencia2: Nivel_Resistencia
    resistencia3: Nivel_Resistencia

class Noticia(BaseModel):
    titulo: str
    link: str 
    resumo: str

class Noticias(BaseModel):
    noticia1: Noticia
    noticia2: Noticia

class Estrategias(BaseModel):
    compradores: str 
    vendedores: str 

class BTCReport(BaseModel):
    resumo: Resumo_de_Mercado
    analise_tecnica: Analise_Tecnica
    suporte_resistencia: Suporte_Resistencia
    noticias: Noticias
    estrategias: Estrategias
    conclusao: str


In [ ]:
BTCReport.estrategias

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field

client = OpenAI()


response = client.responses.parse(
    model="gpt-4.1",
    input=[
        {
            "role": "system",
            "content": prompt_formatado,
        },
    ],
    text_format=BTCReport,
    tools=[{"type": "web_search_preview", "search_context_size": "high"}]
)

resultado = response.output_parsed

In [ ]:
print(resultado)

In [ ]:
json_str = resultado.model_dump_json(indent=2, ensure_ascii=False)
print(json_str)

In [ ]:
response_json = response.model_dump_json(indent=2)
print(response_json)

## Formato Markdown:

In [ ]:
from datetime import date

# ------------------------------------------------------------
# Helpers de formatação numérica em estilo brasileiro
# ------------------------------------------------------------
def _fmt_reais(valor: float, prefixo: str = "R$") -> str:
    """Formata moeda estilo pt-BR: R$ 84.411,00"""
    inteiro, centavos = divmod(round(valor * 100), 100)
    inteiro_fmt = f"{inteiro:,}".replace(",", ".")          # separador de milhar = ponto
    return f"{prefixo} {inteiro_fmt},{centavos:02d}"

def _fmt_grande(valor: float, prefixo: str = "R$") -> str:
    """Converte milhões/bilhões/trilhões mantendo duas casas e devolve string."""
    if valor >= 1e12:
        return f"{_fmt_reais(valor/1e12, prefixo)} trilhões"
    if valor >= 1e9:
        return f"{_fmt_reais(valor/1e9, prefixo)} bilhões"
    if valor >= 1e6:
        return f"{_fmt_reais(valor/1e6, prefixo)} milhões"
    return _fmt_reais(valor, prefixo)

def _pct(valor: float) -> str:
    """Formata porcentagem com duas casas e sinal."""
    return f"{valor:+.2f}%".replace(".", ",")

def _btc(valor: float) -> str:
    """Formata número de BTC em milhões/inteiro."""
    if valor >= 1e6:
        return f"{valor/1e6:.2f}".replace(".", ",") + " milhões BTC"
    return f"{valor:,.0f} BTC".replace(",", ".")

# ------------------------------------------------------------
# Função principal
# ------------------------------------------------------------
def gerar_relatorio_btc_markdown(dados: dict, data_relatorio: date | str | None = None) -> str:
    """
    Gera string Markdown a partir do JSON do BTCReport.
    - dados ........: dict resultante de BTCReport.model_dump()
    - data_relatorio: date ou string; se None usa hoje.
    """
    if data_relatorio is None:
        data_relatorio = date.today()
    if isinstance(data_relatorio, date):
        data_relatorio = data_relatorio.strftime("%d de %B de %Y")  # ex.: 17 de abril de 2025

    # 1. Resumo de Mercado ------------------------------------
    r = dados["resumo"]
    resumo_md = (
f"📌 **Resumo de Mercado - {data_relatorio}**\n"
f"    • Preço Atual: {_fmt_reais(r['preco_atual'])}\n"
f"    • Variação 24h: {_pct(r['variacao_24h_pct'])}\n"
f"    • Máxima 24h: {_fmt_reais(r['max_24h'])}\n"
f"    • Mínima 24h: {_fmt_reais(r['min_24h'])}\n"
f"    • Volume 24h: {_fmt_grande(r['volume_24h'])}\n"
f"    • Capitalização de Mercado: {_fmt_grande(r['market_cap'])}\n"
f"    • Fornecimento Circulante: {_btc(r['fornecimento_circulante'])}\n"
f"    • Fornecimento Máximo: {_btc(r['fornecimento_max'])}\n"
    )

    # 2. Análise Técnica --------------------------------------
    a = dados["analise_tecnica"]
    analise_md = (
f"📊 **Análise Técnica**\n"
f"    • Médias Móveis (MA):\n"
f"        ◦ MA 50 dias: {_fmt_reais(a['sma50'])} ({a['sma50_interpretacao']})\n"
f"        ◦ MA 200 dias: {_fmt_reais(a['sma200'])} ({a['sma200_interpretacao']})\n"
f"        ◦ Cruzamento de Médias: {a['cruzamento_medias_interpretacao']}\n"
f"    • Índice de Força Relativa (RSI):\n"
f"        ◦ Atualmente em {a['rsi']:.0f}, {a['rsi_interpretacao']}\n"
f"    • Bandas de Bollinger:\n"
f"        ◦ {a['bollinger_interpretacao']}\n"
    )

    # 3. Suporte & Resistência --------------------------------
    s = dados["suporte_resistencia"]
    suportes = [s["suporte1"], s["suporte2"], s["suporte3"]]
    resist   = [s["resistencia1"], s["resistencia2"], s["resistencia3"]]

    suportes_md = "\n".join(
        f"        ◦ {_fmt_reais(x['nivel_suporte_valor'])}: {x['nivel_suporte_interpretacao']}"
        for x in suportes
    )
    resist_md = "\n".join(
        f"        ◦ {_fmt_reais(x['nivel_resistencia_valor'])}: {x['nivel_resistencia_interpretacao']}"
        for x in resist
    )
    sr_md = (
f"🔍 **Níveis de Suporte e Resistência**\n"
f"    • Suportes:\n"
f"{suportes_md}\n"
f"    • Resistências:\n"
f"{resist_md}\n"
    )

    # 4. Notícias ---------------------------------------------
    n = dados["noticias"]
    noticias_md = "\n".join(
        f"    • {noticia['titulo']}: {noticia['resumo']}"
        for noticia in n.values()
    )
    noticias_md = f"📰 **Notícias Relevantes**\n{noticias_md}\n"

    # 5. Estratégias ------------------------------------------
    e = dados["estrategias"]
    estrategias_md = (
f"💡 **Considerações Estratégicas**\n"
f"    • Para Compradores:\n"
f"        ◦ {e['compradores']}\n"
f"    • Para Vendedores:\n"
f"        ◦ {e['vendedores']}\n"
    )

    # 6. Conclusão + Aviso ------------------------------------
    conclusao_md = (
f"✅ **Conclusão**\n"
f"    {dados['conclusao']}\n\n"
f"⚠️ **Aviso:** Ao realizar operações de compra ou venda, lembre-se de que o preço do Bitcoin pode sofrer "
f"variações significativas. Faça sempre sua própria análise de risco.\n"
    )

    # Junta tudo e retorna ------------------------------------
    markdown = "\n".join([resumo_md, analise_md, sr_md, noticias_md, estrategias_md, conclusao_md])
    return markdown


In [ ]:
import json
from IPython.display import Markdown, display

data = json.loads(json_str)
md_text = gerar_relatorio_btc_markdown(data)
display(Markdown(md_text))

# 2) Agente de coleta de notícias e analises Tavily:

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field


class Noticia1(BaseModel):
    titulo: str
    link: str
    resumo: str

class Noticia2(BaseModel):
    titulo: str
    link: str
    resumo: str


class Noticia3(BaseModel):
    titulo: str 
    link: str
    resumo: str

class Noticias(BaseModel):
    noticia1: Noticia1
    noticia2: Noticia2
    noticia3: Noticia3


client = OpenAI()


response = client.responses.parse(
    model="gpt-4.1",
    input=[
        {
            "role": "system",
            "content": "Me forneça **três** notícias relevantes encontradas relacionadas ao Bitcoin, incluindo um resumo e o link. As notícias devem ser obrigatoriamente do período de 04/05/2025 até hoje 05/05/2025. Lembre-se, você deve OBRIGATÓRIAMENTE fornecer o titulo da notícia3 APENAS em letras maiúsculas.",
        },
    ],
    text_format=Noticias,
    tools=[{"type": "web_search_preview", "search_context_size": "medium"}]
)

resultado = response.output_parsed

In [ ]:
resultado

In [ ]:
import json
res = (response)
res

# Formato de output desejado:

In [ ]:
"""
📌 Resumo de Mercado – 06 de Junho de 2025
• Preço Atual: US$ 84.411,00
• Variação 24h: +0,62%
• Máxima 24h: US$ 85.312,00
• Mínima 24h: US$ 83.536,00
• Volume 24h: US$ 27,13 bilhões
• Capitalização de Mercado: US$ 1,68 trilhão
• Fornecimento Circulante: 19,85 milhões BTC
• Fornecimento Máximo: 21 milhões BTC


📊 Análise Técnica
• Médias Móveis (MA):
• MA 50 dias: US$ 85.000 (atuando como resistência dinâmica)
• MA 200 dias: US$ 80.000 (servindo como suporte de longo prazo)
• Cruzamento de Médias: Ocorreu recentemente um “cruzamento dourado”, onde a MA de 50 dias cruzou acima da MA de 200 dias, sugerindo uma tendência de alta de longo prazo.
• Índice de Força Relativa (RSI):
• Atualmente em 68, indicando que o ativo está se aproximando da zona de sobrecompra.
• Bandas de Bollinger:
• As bandas estão se estreitando, sinalizando uma redução na volatilidade e possivelmente antecipando um movimento de preço significativo.


🔍 Níveis de Suporte e Resistência
• Suportes:
• US$ 83.900: Suporte imediato observado recentemente.
• US$ 83.061: Próximo nível de suporte técnico.
• US$ 82.478: Alinhado com a média móvel de 200 períodos, servindo como um suporte chave. ￼
• Resistências:
• US$ 84.500 – US$ 84.700: Zona de resistência onde o preço enfrentou rejeições.
• US$ 86.200: Próximo nível de resistência observado.
• US$ 88.600: Resistência adicional a ser monitorada.


📰 Notícias Recentes
• Notícia 1: "Bitcoin atinge novo recorde histórico"
• Link: https://www.binance.com/bitcoin-news
• Resumo: Bitcoin abaixo de US$ 85 mil causa saídas de US$ 171 milhões de ETFs: O Bitcoin tem enfrentado dificuldade significativa em superar a marca de US$ 85 mil nesta semana, e seu preço permanece estagnado abaixo dessa resistência chave. Assim, os entusiastas do BTC estão cada vez mais frustrados. Isso porque a criptomoeda luta para manter o impulso ascendente. Junto com essa estagnação de preço, houve uma queda no interesse aberto e saídas de ETFs, refletindo a crescente incerteza no mercado. 

• Notícia 2: "Adoção institucional do Bitcoin cresce" 
• Link: https://www.binance.com/bitcoin-news
• Resumo: Grandes instituições financeiras estão aumentando suas participações em Bitcoin, sinalizando uma crescente aceitação do ativo digital.


💡 Considerações Estratégicas
• Para Compradores:
• Considerar entradas próximas ao suporte de US$ 83.000, visando uma possível valorização até a resistência de US$ 85.500.
• Para Vendedores:
• Observar a resistência em US$ 85.500 como um possível ponto de realização de lucros, ajustando stops conforme a volatilidade do mercado.


✅ Conclusão
O Bitcoin apresenta uma leve tendência de alta no curto prazo, com suporte em US$ 83.000 e resistência em US$ 85.500. Fatores externos, como políticas econômicas e eventos globais, continuam influenciando o mercado. Recomenda-se monitorar os níveis de suporte e resistência mencionados e estar atento às notícias que possam impactar o valor do ativo.


⚠️ Aviso: Ao realizar operações de compra ou venda, é importante estar ciente de que o preço do Bitcoin pode sofrer variações devido a movimentos de mercado e notícias que possam impactar seu valor.
"""

# Prompts:

## Prompt 1 usando localmente com API:
- Será utilizado com outros agentes, que coletarão as informações necessárias para preencher o relatório
- É uma abordagem mais modular e refinada, pois podemos definir agentes para tarefas especificas, trazendo assim maior grau de refinamento nas respostas.

## Prompt3: usando diretamente no ChatGPT:

In [ ]:
PROMPT3 = f"""
# CONTEXTO
- Você é **BTC_Report_Agent**, analista financeiro especializado em Bitcoin.
- Sua missão é redigir relatórios diários em português, tom analítico, direto e profissional.
- Nunca revele seu raciocínio interno (chain-of-thought).

# REGRAS GERAIS
1. Produza **um único bloco Markdown** seguindo *EXATAMENTE* o “Modelo de Relatório” abaixo.
2. Substitua cada `{{placeholder}}` por seu valor **direto** (número/texto); não mude o formato.
3. Gere cada campo que termina em `_comentario` com **até 2 frases (≤ 25 palavras)**.
4. Inclua, ao final de cada `_comentario`, a fonte entre parênteses usando **[n]** que referencie a seção **Referências**.
5. Saída total ≤ 300 tokens; se necessário, resuma.
6. O aviso (“⚠️ Aviso: …”) é fixo e não deve ser alterado.

# FONTES PERMITIDAS
- **CoinGecko API** (ou similar) para métricas de preço/volume (dados ≤ 30 min).
- **TradingView** (ou similar) para indicadores técnicos.
- **CoinDesk / CoinTelegraph** para notícias publicadas nas últimas 24 h.

# INPUT (preenchido pelo orquestrador antes do LLM)
```yaml
today: "{{today}}"                       # AAAA MM DD
metrics:
  price_current_usd: "{{price_current_usd}}"
  price_change_24h_pct: "{{price_change_24h_pct}}"
  high_24h_usd: "{{high_24h_usd}}"
  low_24h_usd: "{{low_24h_usd}}"
  volume_24h_usd: "{{volume_24h_usd}}"
  market_cap_usd: "{{market_cap_usd}}"
  supply_circulating_btc: "{{supply_circulating_btc}}"
  supply_max_btc: "{{supply_max_btc}}"
  ma_50_usd: "{{ma_50_usd}}"
  ma_200_usd: "{{ma_200_usd}}"
  rsi_value: "{{rsi_value}}"
  supports: [{{supports_0}}, {{supports_1}}, {{supports_2}}]
  resistances: [{{resistances_0}}, {{resistances_1}}, {{resistances_2}}]
  headline1: "{{headline1}}"
  headline2: "{{headline2}}"
```

# TAREFAS
- Preencher o “Modelo de Relatório” replicando texto fixo + valores diretos + análises nos campos `_comentario`.
- Para `headline1` e `headline2`, gerar `noticiaX_comentario` ≤ 2 frases cada (impacto no mercado).

# OUTPUT FORMAT
```markdown
📌 **Resumo de Mercado - {{today}}**  
• Preço Atual: US$ {{price_current_usd}}  
• Variação 24h: {{price_change_24h_pct}} %  
• Máxima 24 h: US$ {{high_24h_usd}}  
• Mínima 24 h: US$ {{low_24h_usd}}  
• Volume 24 h: US$ {{volume_24h_usd}}  
• Capitalização de Mercado: US$ {{market_cap_usd}}  
• Fornecimento Circulante: {{supply_circulating_btc}} mi BTC  
• Fornecimento Máximo: {{supply_max_btc}} mi BTC  

📊 **Análise Técnica**  
• MA 50 d: US$ {{ma_50_usd}} — {{ma_50_comentario}}  
• MA 200 d: US$ {{ma_200_usd}} — {{ma_200_comentario}}  
• Cruzamento de Médias: {{cruzamento_medias_comentario}}  
• RSI ({{rsi_value}}): {{rsi_comentario}}  
• Bandas de Bollinger: {{bollinger_comentario}}  

🔍 **Níveis de Suporte e Resistência**  
• **Suportes**  
  1. US$ {{supports_0}} — {{supports0_comentario}}  
  2. US$ {{supports_1}} — {{supports1_comentario}}  
  3. US$ {{supports_2}} — {{supports2_comentario}}  
• **Resistências**  
  1. US$ {{resistances_0}} — {{resist0_comentario}}  
  2. US$ {{resistances_1}} — {{resist1_comentario}}  
  3. US$ {{resistances_2}} — {{resist2_comentario}}  

📰 **Notícias Recentes**  
• {{headline1}}  
  - {{noticia1_comentario}}  
• {{headline2}}  
  - {{noticia2_comentario}}  

💡 **Considerações Estratégicas**  
• Para Compradores: {{estrategia_compradores_comentario}}  
• Para Vendedores: {{estrategia_vendedores_comentario}}  

✅ **Conclusão**  
{{conclusao_comentario}}  

⚠️ **Aviso**  
⚠️ Aviso: Ao colocar uma ordem de compra ou venda, é importante estar ciente de que o preço futuro do Bitcoin pode sofrer variações devido a movimentos de mercado de última hora e notícias que possam impactar seu valor.
```
"""

## Usando CoinMarketCap API:

https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyMarketpairsLatest

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint
import os
from dotenv import load_dotenv
load_dotenv()

url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'

parameters = {
    "symbol": "BTC",
    "convert": "USD",
    #"interval": "1d",
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': os.getenv('COIN_MARKET_CAP_API_KEY'),
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

## Usando Yahoo Finance:

https://yfinance-python.org/reference/api/yfinance.download.html#yfinance.download

## Ferramenta de busca Tavily

In [ ]:
from tavily import TavilyClient
from dotenv import load_dotenv
_ = load_dotenv()

tavily_client = TavilyClient()

In [ ]:
topic = "Bitcoin"
context = []
search_response = tavily_client.search(topic, topic="finance", search_depth="advanced", time_range="day", max_results=3)
# context.append(
#     {
#         "url": search_response["results"]["url"],
#         "title": search_response["results"]["title"],
#         "content": search_response["results"]["content"],
#     }
# )

In [ ]:
search_response

In [ ]:
#print(response["results"])
resp = search_response["results"][1]['content']
print(resp)

In [ ]:
# Função que formata a resposta da busca de noticias em um dicionário
def extrair_dados_noticias(api_response):
    i = 1
    for noticia in api_response['results']:
        print(f"Notícia {i}:")
        print("Título: ", noticia['title'])
        print("URL: ", noticia['url'])
        print("Data: ", noticia['published_date'])
        print("Conteúdo: ", noticia['content'])
        print("\n")
        i += 1

def format_news(data: dict) -> str:
    results = data.get('results', [])
    lines = []
    for idx, item in enumerate(results, start=1):
        title = item.get('title', '').strip()
        url = item.get('url', '').strip()
        date = item.get('published_date', '').strip()
        content = item.get('content', '').strip()

        lines.append(f"Notícia {idx}:")
        lines.append(f"Título: {title}")
        lines.append(f"URL: {url}")
        lines.append(f"Data de publicação: {date}")
        lines.append(f"Conteúdo: {content}")
        lines.append("")  # linha em branco entre notícias

    return "\n".join(lines)

In [ ]:
extrair_dados_noticias(search_response)

In [ ]:
news = format_news(search_response)
print(news)

------------------------------

In [ ]:
import yfinance as yf
symbol = "BTC-USD"  # Bitcoin vs USD
# Coletar dados usando yfinance
raw_data = yf.Ticker(symbol)
raw_data_dict = raw_data.get_news(count=5, tab="news")
raw_data_dict

In [ ]:
raw_data.news

In [ ]:
raw_data_dict[0]['content']['summary']

In [ ]:
def extrair_dados_yahoo(raw_data_dict):
    i = 1
    for noticia in raw_data_dict['content']:
        print(f"Notícia {i}:")
        print("Título: ", noticia['title'])
        print("URL: ", noticia['url'])
        print("Data: ", noticia['pubDate'])
        print("Conteúdo: ", noticia['summary'])
        print("\n")
        i += 1